In [18]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import keras
import os


In [26]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(device_count={'GPU': 0}))

In [27]:
df = pd.read_csv('df_clean.csv')
df = df.iloc[:,3:]

In [28]:
#remove mentions, urls, hashtags, ;&, and 'rt' and other punctuation. keep a count of mentions, urls, hashtags
tweets = df['tweet_lemma']

mentions = []
urls = []
hashtags = []
i = 0
for tweet in tweets:
    tweet = tweet.split()
    mentions.append(tweet.count('mentionhere')+tweet.count('mentionhere:')+tweet.count('"mentionhere:')+tweet.count('&#;mentionhere:'))
    urls.append(tweet.count('urlhere'))
    hashtags.append(tweet.count('hashtaghere'))
    tweet = [token for token in tweet if token not in [';&','']]
    tweet = [token for token in tweet if token not in ['&#;mentionhere:','mentionhere:','"mentionhere:','mentionhere', 'urlhere', 'hashtaghere', 'rt', 'amp']]
    tweet = " ".join(tweet)
    tweets[i] = tweet
    i += 1
    
df['tweet_no_others'] = tweets
df['mention_count'] = mentions
df['url_count'] = urls
df['hashtag_count'] = hashtags

df.head()

C:\Users\seanx\anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


,count,hate_speech,offensive_language,neither,class,tweet,tweet_clean,tweet_lemma,tweet_nouns,tweet_sym,tweet_verbs,tweet_nv,num_tokens,tweet_no_others,mention_count,url_count,hashtag_count
0,3,1,2,0,1,""" bitch who do you love """,bitch who do you love,bitch who do -PRON- love,bitch,NaN,love,bitch love,5.0,bitch who do -PRON- love,0,0,0
1,3,0,3,0,1,""" fuck no that bitch dont even suck dick "" &#1...",fuck no that bitch dont even suck dick ...,fuck no that bitch do not even suck dick the k...,bitch dick kermit video bout,NaN,suck fuck,bitch dick kermit video bout suck fuck,18.0,fuck no that bitch do not even suck dick the k...,0,0,0
2,3,0,2,1,1,""" lames crying over hoes thats tears of a clown """,lames crying over hoes thats tears of a clown,lame cry over hoe that s tear of a clown,lame hoe tear clown,NaN,cry s,lame hoe tear clown cry s,10.0,lame cry over hoe that s tear of a clown,0,0,0
3,3,0,3,0,1,"""..All I wanna do is get money and fuck model ...",all i wanna do is get money and fuck model bit...,all i wanna do be get money and fuck model bit...,wanna money fuck model bitch russell simmons,NaN,NaN,wanna money fuck model bitch russell simmons,14.0,all i wanna do be get money and fuck model bit...,0,0,0
4,3,0,3,0,1,"""@ARIZZLEINDACUT: Females think dating a pussy...",mentionhere females think dating a pussy is c...,female think date a pussy be cute now how do d...,mentionhere female pussy stuff pussy,NaN,think date urlhere do make,mentionhere female pussy stuff pussy think dat...,22.0,female think date a pussy be cute now how do d...,1,1,0


In [29]:
feature_array = pd.read_csv('twitter_feature_array.csv')
feature_array = feature_array.iloc[:,1:]
feature_array.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.092240,0.655190,-0.054113,-0.001526,0.245873,-0.047122,-0.195120,-0.241070,-0.024567,-0.028244,...,-0.009025,0.098637,0.100423,0.644974,-0.122191,-0.027687,-0.357734,-0.078975,0.641340,-0.015340
1,-0.036190,0.052367,0.056565,0.040076,0.153519,-0.078184,-0.050946,-0.083378,-0.090013,0.050202,...,0.086134,-0.051438,0.051010,0.250482,-0.081903,0.051434,-0.121692,-0.058639,0.217542,-0.000544
2,-0.084097,0.030291,0.036536,0.039797,0.106109,0.002597,0.060696,-0.047483,-0.047069,0.039118,...,0.095309,-0.026714,0.062526,0.106153,-0.041362,0.022510,-0.119272,0.038568,0.058572,0.055155
3,0.007605,0.010492,0.032044,0.126457,0.047635,-0.069211,-0.054946,-0.011408,-0.095851,0.020046,...,0.103008,-0.043711,0.106259,0.163596,0.011475,0.101757,-0.065334,-0.043148,0.092530,0.035289
4,-0.100222,-0.060809,0.076636,-0.023275,0.134729,-0.025185,0.020398,-0.055950,-0.117073,-0.033665,...,0.097490,-0.023263,0.055456,0.143194,-0.056619,0.058868,-0.130279,-0.070994,0.091588,0.107159


In [38]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(5, input_dim=100, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(5, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 5)                 505       
_________________________________________________________________
dense_14 (Dense)             (None, 5)                 30        
_________________________________________________________________
dense_15 (Dense)             (None, 2)                 12        
Total params: 547
Trainable params: 547
Non-trainable params: 0
_________________________________________________________________
None


In [39]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

X = feature_array
y = to_categorical(df['class'])



In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

In [41]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, verbose=0)

InternalError: cudaGetDevice() failed. Status: CUDA driver version is insufficient for CUDA runtime version

In [42]:
nvidia-smi

NameError: name 'nvidia' is not defined